In [ ]:
import pandas as pd
import datetime
from ipywidgets import *
pd.set_option('display.max_colwidth', 128)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 60)

In [ ]:
"""
-- Modified original query from https://dune.com/queries/92408/184718

SELECT 
  tx.hash,
  tx.success,
  --pid."name", 
  mints."_projectId" AS ProjectID,
  tx.value/1e18 AS price_eth,
  date_trunc('second', mints."evt_block_time") AS time,   
  mints."_to" AS buyer, 
  (tx."gas_used" * tx."gas_price"/1e18) AS gas_eth
FROM artblocks."GenArt721_evt_Mint" mints -- old contrct
LEFT JOIN ethereum.transactions tx
  ON mints."evt_tx_hash" = tx."hash"
--LEFT JOIN dune_user_generated.ArtBlocksProjectIDs pid 
--  ON pid.id = mints."_projectId"

UNION ALL 
    
SELECT 
  tx.hash,
  tx.success,
  --pid."name", 
  mints."_projectId" AS ProjectID, 
  tx.value/1e18 AS price, 
  date_trunc('second', mints."call_block_time") AS time, 
  mints."_by" AS buyer, 
  (tx."gas_used" * tx."gas_price"/1e18) AS gas_eth
FROM artblocks."GenArt721Core_call_mint" mints -- new contract
LEFT JOIN ethereum.transactions tx
  ON mints."call_tx_hash" = tx."hash"
--LEFT JOIN dune_user_generated.ArtBlocksProjectIDs pid 
--  ON pid.id = mints."_projectId"
WHERE "output__tokenId" is not null
ORDER BY time DESC
"""

d = pd.read_csv('../mint.csv')
d["time"] = pd.to_datetime(d["time"])
display(d.dtypes)

# sort by time and descending gas for most probable execution order without looking at transaction order numbers.
d.sort_values(by=["time", "gas_eth"], ascending=[True, False], inplace=True)

d.head()

In [ ]:
display(d.info())
display("Number of successful mints: ", d.success.sum())
d.describe(include=['bool','float', 'int', 'datetime'])

In [ ]:
types = pd.read_csv('../typebyname.csv')
display(types.info())

In [ ]:
mints = d[d["success"]]

import statistics

def getMiddleValue(pdSeries):
    mid = (pdSeries.count() / 2).astype(int)
    return pdSeries.iloc[mid]

types = types.set_index("projectid")
types = types["project_type"]
types = types.reset_index()
# adding project type to mint dataset
mints = pd.merge(mints, types, on="projectid", how="outer")
mintsByProjectId = mints.groupby("projectid")
mintsByProjectId = pd.DataFrame({
    "count": mintsByProjectId["projectid"].count(),
    "firstMintTime": mintsByProjectId["time"].first(),
    "lastMintTime": mintsByProjectId["time"].last(),
    "lastMintPriceTotal": mintsByProjectId["price_eth"].last() + mintsByProjectId["gas_eth"].last(),
    "minMintPrice": mintsByProjectId["price_eth"].min(),
	"medianMintPrice": mintsByProjectId["price_eth"].median(),
	"meanMintPrice": mintsByProjectId["price_eth"].mean(),
    "medianMintTime":  mintsByProjectId["time"].apply(lambda x: getMiddleValue(x)),
    "projectType": mintsByProjectId["project_type"].first()
    })
mintsByProjectId["latterMintWindowInMins"] = round((mintsByProjectId["lastMintTime"] - mintsByProjectId["medianMintTime"]).dt.total_seconds() / 60, 2)
mintsByProjectId["totalMintWindowInMins"] = ((mintsByProjectId["lastMintTime"] - mintsByProjectId["firstMintTime"]).dt.total_seconds() / 60)
mintsByProjectId["latterMintWindowUnder4Hours"] = (mintsByProjectId["latterMintWindowInMins"] <= 240)

In [ ]:
mintsByProjectIdNoIndex = mintsByProjectId.reset_index()
mintsByProjectIdNoIndex = mintsByProjectIdNoIndex[mintsByProjectIdNoIndex["projectType"] == "Curated"]
settings = ["medianMintPrice", "meanMintPrice", "minMintPrice", "totalMintWindowInMins"]
def updateStats(i = 1):
	mintsByProjectIdNoIndex.plot.scatter(x="projectid", y=settings[i], figsize=(20,8), title="curated collections")
	mintsByProjectIdNoIndex[mintsByProjectIdNoIndex["latterMintWindowUnder4Hours"] == True].plot.scatter(x="projectid", y=settings[i], figsize=(20,8), title="latter mint window under 4 hours")

interact(updateStats)

In [ ]:
"""
-- Modified original query from https://dune.com/queries/160701/314169

select distinct block_time, 
  ROUND("nft_token_id"::numeric / 1000000) as projectid,
  round(eth_amount, 2) as eth_price, 
  usd_price, 
  link, 
  platform, 
  left(seller::text, 7) as seller, 
  left(buyer::text, 7) as buyer 
from 
(
select 
  block_time, 
  platform, 
  usd_amount, 
     
  case 
     when ("original_currency" = 'ETH' OR "original_currency" = 'WETH')
             THEN  ("original_amount")
    else 0  
  END as eth_amount, 
  "usd_amount" as usd_price,

   
 CONCAT('<a href="https://opensea.io/assets/', CONCAT('0x', substring(a."nft_contract_address"::text from 3)), '/', a.nft_token_id,  '/?ref=0x8F903cFC0Af3C2EC0d872c57538AF5e071544a57','" target="_blank" >', 'View on OS', '</a>') as  link,
   
 seller, 
 buyer, 
 tx_hash,
 nft_token_id

from nft."trades" a
WHERE 
     "trade_type" = 'Single Item Trade'
     AND (a.nft_contract_address = '\xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270'
    OR  a.nft_contract_address = '\x059edd72cd353df5106d2b9cc5ab83a52287ac3a')
ORDER BY block_time DESC 
) gg
-- WHERE block_time > '{{Date}}'
order by block_time DESC
"""

p = pd.read_csv('../sales.csv')

display("Before filtering:", len(d))

# cleaning up weird project ids
#p_removed = p[p["projectid"].str.len() >= 8]
#p = p[p["projectid"].str.len() < 8]

# casting
p["time"] = pd.to_datetime(p["time"])
p["projectid"] = p["projectid"].astype(int)

p.sort_values(by=["time"], ascending=[True], inplace=True)

#removing non valid transactions
p = p[p["eth_total"] > 0]

# adding derived data
p["normalized_price"] = p["eth_total"] / p["projectid"].map(mintsByProjectId.lastMintPriceTotal)
p["latterMintWindowInMins"] = p["projectid"].map(mintsByProjectId.latterMintWindowInMins)
p["lastMintTime"] = p["projectid"].map(mintsByProjectId.lastMintTime)
p["isWithin2hFromLastMintTime"] = ((p["time"] - p["lastMintTime"]).dt.total_seconds() / 60 < 120) & ((p["time"] - p["lastMintTime"]).dt.total_seconds() > 0)

display(p.dtypes)
display(p.describe(include=['bool','float', 'int', 'datetime']))
p

In [ ]:
tradesByProjectId = p.groupby("projectid")
tradesByProjectId = pd.DataFrame({
    "tradeCount": tradesByProjectId["projectid"].count(),
    "tradeCount2hr": tradesByProjectId["isWithin2hFromLastMintTime"].sum(),
    "medianNormPrice2h": tradesByProjectId.apply(lambda df: df[df["isWithin2hFromLastMintTime"]].normalized_price.median()),
    "meanNormPrice2h": tradesByProjectId.apply(lambda df: df[df["isWithin2hFromLastMintTime"]].normalized_price.mean()),
    "projectType": tradesByProjectId["project_type"].first(),
})
tradesByProjectId = tradesByProjectId.reset_index()
tradesByProjectId["latterMintWindowInMins"] = tradesByProjectId["projectid"].map(mintsByProjectId.latterMintWindowInMins)

# removing 0 sales projects -> very old ones
tradesByProjectId = tradesByProjectId[tradesByProjectId["tradeCount2hr"] > 0]

display("number of collections", len(tradesByProjectId.index))
groupedTradesByProjectId = tradesByProjectId.groupby("projectType")
tradesStats = pd.DataFrame({
    "averageTradesWithin2hr": groupedTradesByProjectId["tradeCount2hr"].mean(),
    "minTradesWithin2hr": groupedTradesByProjectId["tradeCount2hr"].min(),
    "maxTradesWithin2hr": groupedTradesByProjectId["tradeCount2hr"].max(),
    "medianTradesWithin2hr": groupedTradesByProjectId["tradeCount2hr"].median(),
})
tradesStats

In [ ]:
def update(projectid = 331):
    pId = p[p["projectid"] == projectid]
    display("project type: ", pId["project_type"].iloc[0])
    pId[pId["isWithin2hFromLastMintTime"]].plot(x="time", y=["normalized_price", "eth_total"], figsize=(20,8))

interact(update)

In [ ]:
import matplotlib.pyplot as plt
medianMintWindow = tradesByProjectId["latterMintWindowInMins"].median()
firstQuantileMintWindow = tradesByProjectId["latterMintWindowInMins"].quantile(.25)
thirdQuantileMintWindow = tradesByProjectId["latterMintWindowInMins"].quantile(.75)
maxMintWindow = tradesByProjectId["latterMintWindowInMins"].max()
mintWindowValues = [firstQuantileMintWindow, medianMintWindow, thirdQuantileMintWindow, maxMintWindow]
collectionType = ["Curated", "Playground", "Factory"]
colors = ["green", "blue", "red"]
def updateWindow(i = 1):
    fig, axs = plt.subplots(2)
    displayedTrades = tradesByProjectId[tradesByProjectId["latterMintWindowInMins"] <= mintWindowValues[i]]

    # removing high profit collection to get a better view on more "casual" collections
    displayedTrades = displayedTrades[displayedTrades["medianNormPrice2h"] < 3]

    print("number of collections:", displayedTrades["projectType"].count())
    for idx,v in enumerate(collectionType):
        print(colors[idx])
        print(v)
        x = displayedTrades[displayedTrades["projectType"] == v].latterMintWindowInMins
        y = displayedTrades[displayedTrades["projectType"] == v].tradeCount2hr
        y2 = displayedTrades[displayedTrades["projectType"] == v].medianNormPrice2h
        axs[0].set_xlabel("mint window(minutes)")
        axs[0].set_ylabel("trade count within 2 hours after last mint")
        axs[1].axhline(y=1, color='r', linestyle='-')
        axs[1].set_xlabel("mint window(minutes)")
        axs[1].set_ylabel("median normalised price within 2 hours after last mint")
        axs[0].scatter(x, y, color=colors[idx])
        axs[1].scatter(x, y2, color=colors[idx])

interact(updateWindow)

In [ ]:
from datetime import timezone, timedelta, datetime
def updateProfit(mintWindow = 120, withinMins = 120, minDate=100, maxDate=100):
    #creating custom time selector in the main dataset
    p["isWithinxFromLastMintTime"] = ((p["time"] - p["lastMintTime"]).dt.total_seconds() / 60 < withinMins) & ((p["time"] - p["lastMintTime"]).dt.total_seconds() > 0)
    today = datetime.now(timezone.utc)
    maxDelta = timedelta(days=maxDate)
    minDelta = timedelta(days=minDate)
    p["isRecent"] = (today - p["lastMintTime"]) < maxDelta
    p["isRecent"] = (today - p[p["isRecent"] == True].lastMintTime) > minDelta
    
    #adding it to my tradesByProjectId set
    pgrouped = p.groupby("projectid")
    pgrouped = pd.DataFrame({
        "withinXMins": pgrouped.apply(lambda df: df[df["isWithinxFromLastMintTime"]].normalized_price.median()),
        "isRecent": pgrouped["isRecent"].first(),
    })
    tradesByProjectId["withinXMins"] = tradesByProjectId["projectid"].map(pgrouped.withinXMins)
    tradesByProjectId["isRecent"] = tradesByProjectId["projectid"].map(pgrouped.isRecent)

    #selecting the matching latterMintWindow
    soldOut = tradesByProjectId[tradesByProjectId["latterMintWindowInMins"] <= mintWindow]
    soldOut = soldOut[soldOut["isRecent"] == True]
    soldOut = soldOut[soldOut["projectType"] == "Curated"]

    #displaying data
    print("number of collections:", soldOut["projectid"].count())
    profits = soldOut[soldOut["withinXMins"] >= 1.1].projectid.count()
    loss = soldOut[soldOut["withinXMins"] < 1.1].projectid.count()
    print("for collections with a latterMintWindow <=", mintWindow, "mins")
    print("(based on median price within", withinMins, "mins after last mint time)")
    print("collections released from: ", today - maxDelta, " to: ", today - minDelta)
    print("number of profits:", profits)
    print("number of losses:", loss)
    print("profit rate(%):", profits / (profits + loss) * 100)

interact(updateProfit)

In [ ]:
def updateProfitWindow(mintWindow = 120, startWindow = 60, endWindow = 70):
    #creating custom time selector in the main dataset
    p["isWithinWindowAfterMint"] = ((p["time"] - p["lastMintTime"]).dt.total_seconds() / 60 > startWindow) & ((p["time"] - p["lastMintTime"]).dt.total_seconds() / 60 < endWindow)
    
    #adding it to my tradesByProjectId set
    pgrouped = p.groupby("projectid")
    pgrouped = pd.DataFrame({
        "withinWindow": pgrouped.apply(lambda df: df[df["isWithinWindowAfterMint"]].normalized_price.median()),
    })
    tradesByProjectId["withinWindow"] = tradesByProjectId["projectid"].map(pgrouped.withinWindow)

    #selecting the matching latterMintWindow
    soldOut = tradesByProjectId[tradesByProjectId["latterMintWindowInMins"] <= mintWindow]

    #displaying data
    print("number of collections:", soldOut["projectid"].count())
    profits = soldOut[soldOut["withinWindow"] >= 1.1].projectid.count()
    loss = soldOut[soldOut["withinWindow"] < 1.1].projectid.count()
    print("for collections with a latterMintWindow <=", mintWindow, "mins")
    print("(based on median price within", startWindow, "mins after last mint time to", endWindow, "mins after mint)")
    print("number of profits:", profits)
    print("number of losses:", loss)
    print("profit rate(%):", profits / (profits + loss) * 100)

interact(updateProfitWindow)